# NOC processing
#### This is the fourth and final step of the enrichment calculation
Calculate NOC proportion and append to the enrichment table  

Key output files
- `yyyy-mm-dd_enrichment_table_NOC_prop.csv`: enrichment values and NOC proportion  
- `yyyy-mm-dd_volcano_table.csv`: enrichment values and p-values  
  
both files are in the directory `output/enrichment_and_volcano_tables`

In [8]:
import os, sys
from pathlib import Path
import pandas as pd

script_path = Path.cwd().parent.parent / "script"
data_path = Path.cwd().parent.parent / "data"
sys.path.append(str(script_path))

from pyseus import basic_processing as ip
from pyseus import contrast_tools as ct

### Load imputed NOC/Bulk table
The correct datestamp is required to find the input files

In [9]:
%store -r timestamp USE_FROZEN
if USE_FROZEN:
    raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from Fig1")
print(f"Timestamp: {timestamp}")
outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

NOC_path = preprocessing_out / f"{outprefix}_filtered_table.csv"

try:
    noc_table = pd.read_csv(NOC_path, header=[0], index_col=0)
except FileNotFoundError:
    print(f"File {NOC_path} not found.\nPlease rerun the previous steps or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {NOC_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Timestamp: 2024-10-26


### Calculation of proportion

In [10]:
# take only the noc columns
NOC_kw = ["Cytosol", "Nuclear", "Organelle"]
noc_cols = [
    col for col in list(noc_table)
    if (any(kw in col for kw in NOC_kw)) and "Proteome" not in col
]

In [11]:
# select the noc_cols + Protein IDs for merge purpose
noc_table = noc_table[['Protein IDs'] + noc_cols].copy()

In [12]:
# group the tables and take the median

# start RawTables class from basic processing to group the samples
noc_process = ip.RawTables(proteingroup=None, file_designated=True,
    info_cols=['Protein IDs'], sample_cols=noc_cols)

# designate the pre-computed noc filtered table on this class
noc_process.filtered_table = noc_table

# transform the intensity to linear space
#noc_process.transform_intensities(func=np.exp2)

# group replicates
noc_process.group_replicates(reg_exp=r'(.*)_\d+$')

# calculate the median
noc_medians = ip.median_replicates(noc_process.grouped_table)
noc_medians.head()

Intensity values have not been transformed yet from filtered table,
we recommend using transform_intensities() method before grouping replicates.

Using filtered_table to group replicates.


,UnInfected_Nuclear,UnInfected_Cytosol,UnInfected_Organelle,Protein IDs
2,166520000.0,75060000.0,76811000.0,A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124
3,238020000.0,245940000.0,194700000.0,Q9Y5S9;A0A023T787;A0A0J9YW13
5,22978000.0,4330700.0,51652000.0,A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...
6,0.0,3272600.0,0.0,A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...
7,24210000.0,9258400.0,77290000.0,Q99805;A0A024QYR8;B3KSG9


In [13]:
# we then take the sum of each column 
noc_medians['noc_sum_uninf'] = noc_medians['UnInfected_Cytosol'] + noc_medians['UnInfected_Organelle'] + noc_medians['UnInfected_Nuclear']

# remove all proteingroups where both inf and uninf noc_sum is 0 -> there is no information on these
noc_medians = noc_medians[(noc_medians['noc_sum_uninf'] > 0)].reset_index(drop=True)

# create a new dataframe with just the proportion: intensity divided by the sum
noc_proportion = noc_medians[['Protein IDs']].copy()
noc_proportion['NOC_cytosol'] = noc_medians['UnInfected_Cytosol'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_organelle'] = noc_medians['UnInfected_Organelle'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_nuclear'] = noc_medians['UnInfected_Nuclear'] / noc_medians['noc_sum_uninf']


### Merge NOC proportion with IP enrichment

In [14]:
# merge with the enrichment table
# read the enrichment table
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
enrichment_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table.csv"
enrichments = pd.read_csv(enrichment_csv_path, header=[0, 1], index_col=0)
meta_cols = enrichments["metadata"].columns
enrichments = enrichments.droplevel(0, axis=1)

In [15]:
# merge with the enrichment table
enrichment_nocs = enrichments.merge(noc_proportion, on="Protein IDs", how="inner")

# standard formatting
enrichment_nocs = ct.standard_pyseus_headers(enrichment_nocs, meta_cols=meta_cols)

In [16]:
# save the enrichment table with NOC proportions
enrichment_NOC_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table_NOC_prop.csv"
enrichment_nocs.to_csv(enrichment_NOC_csv_path)

In [17]:
# view the enrichment table with NOC proportions
enrichment_nocs

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8536                                             X5D7P8   
8537                                             X5D8X9   
8538  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8539                                             X5DQZ7   
8540                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8536                                             X5D7P8            STK39   
8537                                             X5D8X9          CNTNAP2   
8538  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8539                                             X5DQZ7             GPX1   
8540                                             X6RHX1            SCYL3   

        sample                                                              \
     13-GOLGA2 12-PNPLA2 12-SEC61B     12-WT  13-RAB14  12-LAMP1  13-RAB7A   
0    -0.832567  0.590713 -0.265127 -0.829845  0.218249  0.006886 -0.538597   
1    -0.430800  0.677400 -0.631700  1.425600 -0.984800 -0.588500 -1.357550   
2     6.283809 -4.335088  4.012383 -3.924100  4.320300  3.148130  4.496779   
3     0.739701 -0.845441  0.848930 -0.035370 -0.035675  0.088362  0.075128   
4     6.830250 -8.207671  6.156061 -5.951607  7.166215  5.852807  8.552372   
...        ...       ...       ...       ...       ...       ...       ...   
8536 -0.471505 -0.867377 -0.384673  1.035184 -0.725610 -0.089538 -0.244117   
8537  0.919475  0.094208 -0.425143  0.300054  0.481218 -0.368300 -0.933941   
8538 -0.808379 -0.123037  0.274291 -0.414747  0.292167  0.734020  0.619879   
8539 -0.961491 -0.766564  0.437071 -0.238496  0.106310  0.130053  0.224640   
8540  0.230569 -0.192520 -0.826784  0.296719  0.247025  0.462630  0.099394   

      ...                                                                \
      ...     09-WT 09-HSP90AA1  09-PSMB7  10-AP2B1     10-WT 11-CEP350   
0     ...  2.089331   -0.768255 -0.628961 -0.824471  2.777331 -0.213170   
1     ...  1.836900   -2.443700 -0.327100 -0.336600  1.972000  0.895300   
2     ...  0.315600   -4.311600 -3.643343  0.620900  0.006200  0.397300   
3     ...  0.085321   -0.052780  0.522294 -0.755410 -0.056601  0.233463   
4     ... -0.744500   -3.389100 -1.587400  1.823800 -0.608600 -0.438800   
...   ...       ...         ...       ...       ...       ...       ...   
8536  ... -0.297478   -0.446006 -0.220150 -0.297105 -0.177771  0.846096   
8537  ... -0.635438   -0.468167 -0.489276 -1.360433 -0.010147  2.103270   
8538  ...  0.154687   -0.534041 -0.057126  0.138068  0.263834  0.939305   
8539  ... -0.156213    3.753081  0.060683  0.283482 -0.283292  0.831158   
8540  ... -0.041049   -0.302357 -0.992837 -0.201681  0.031179  0.451126   

                                                      
     10-TOMM20 NOC_cytosol NOC_organelle NOC_nuclear  
0    -0.559929    0.235748      0.241247    0.523005  
1    -2.969400    0.362391      0.286889    0.350721  
2    -2.685600    0.054846      0.654148    0.291006  
3     0.841546    1.0000